In [1]:
import numpy as np
import sys
sys.path.append('..')
from custom_lib import quantum_circuit_ctx_generator, utils, verilog, fixed_point_handler

In [16]:
def quantum_circuit_computation_local(n_qubit=3, n_quanv=1):
    ctx_link = '../hardware/gate_ctx_for_sim/quantum_circuit_data_' + str(n_qubit) + '_qubits/' + 'output_' + str(n_qubit) + '_qubits' + '_quanvolutional_' + str(n_quanv) + '.txt'
    gate_ctx_data = utils.read_file(ctx_link)

    I_matrix = np.array([[1, 0], [0, 1]], dtype=complex)
    Dense_matrix = np.array([[0, 0], [0, 0]], dtype=complex)

    state_vector = np.zeros(2**n_qubit, dtype=complex)
    state_vector[0] = 1

    bit_width = 32
    fraction_width = 30

    sub_idx = 0
    gate_type = 1 # 0: Sprase, 1: Dense, 2: CX
    num_of_gates = 0
    gate_count = 0
    for index, line in enumerate(gate_ctx_data):
        if index == 0: # Number of gates
            num_of_gates = int(line.strip())
            print(f"=> Number of gates: {num_of_gates}")
        else:
            if(sub_idx == 0): # Process gate type
                print(f"==================== Gate {gate_count} ====================")
                gate_count = gate_count + 1
                gate_type = int(line.strip())
                sub_idx = sub_idx + 1
            elif(sub_idx == 1):
                if(gate_type == 1): # Process dense gate position
                    pos = n_qubit-1-int(line.strip())
                    # pos = int(line.strip())
                    print(f"====> Dense gate | Pos: {pos}")
                    sub_idx = sub_idx + 1
                elif(gate_type == 2): # Process CX gate position
                    sub_idx = 0
                    ctrl_pos, tgt_pos = line.strip("{}").split(", ")
                    ctrl_pos = n_qubit-1-int(ctrl_pos)
                    tgt_pos = n_qubit-1-int(tgt_pos)
                    # ctrl_pos = int(ctrl_pos)
                    # tgt_pos = int(tgt_pos)

                    print(f"====> CX gate | Ctrl Pos: {ctrl_pos} | Tgt Pos: {tgt_pos}")

                    cx_matrix = utils.cx_matrix_formula(n_qubit, ctrl_pos, tgt_pos)
                    state_vector = np.dot(cx_matrix, state_vector)

                    print(f"State vector after CX gate:")
                    for i in range(2**n_qubit):
                        real_part = state_vector[i].real
                        imaginary_part = state_vector[i].imag

                        real_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=real_part, bit_width=bit_width, dot_position=fraction_width)
                        imaginary_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=imaginary_part, bit_width=bit_width, dot_position=fraction_width)

                        real_part_hex = fixed_point_handler.binary_to_hex(binary_str=real_part_binary, bit_width=bit_width)
                        imaginary_part_hex = fixed_point_handler.binary_to_hex(binary_str=imaginary_part_binary, bit_width=bit_width)
                        if(i == 5 or i == 6):
                            print(f"Complex number: {state_vector[i]} | Hex form: {real_part_hex[2:]+imaginary_part_hex[2:]}")
            else: # Process dense gate entries
                if(gate_type):
                    if(line[0] == '('):
                        line = line.strip("()")
                        
                    line = line.strip("")
                    compl_num = complex(line)

                    if(sub_idx == 2):
                        Dense_matrix[0][0] = compl_num
                    elif(sub_idx == 3):
                        Dense_matrix[0][1] = compl_num
                    elif(sub_idx == 4):
                        Dense_matrix[1][0] = compl_num
                    elif(sub_idx == 5):
                        Dense_matrix[1][1] = compl_num

                        full_dense_matrix = utils.create_full_gate_matrix(gate_matrix=Dense_matrix, gate_pos=pos, n_qubit=n_qubit)
                        state_vector = np.dot(full_dense_matrix, state_vector)
                        print(f"State vector after Dense gate:")
                        for i in range(2**n_qubit):
                            real_part = state_vector[i].real
                            imaginary_part = state_vector[i].imag

                            real_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=real_part, bit_width=bit_width, dot_position=fraction_width)
                            imaginary_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=imaginary_part, bit_width=bit_width, dot_position=fraction_width)

                            real_part_hex = fixed_point_handler.binary_to_hex(binary_str=real_part_binary, bit_width=bit_width)
                            imaginary_part_hex = fixed_point_handler.binary_to_hex(binary_str=imaginary_part_binary, bit_width=bit_width)
                            if(i == 5 or i == 6):
                                print(f"Complex number: {state_vector[i]} | Hex form: {real_part_hex[2:]+imaginary_part_hex[2:]}")

                    if(sub_idx < 5):
                        sub_idx = sub_idx + 1
                    else:
                        sub_idx = 0

In [17]:
# utils.quantum_circuit_computation(n_qubit=10, n_quanv=3)
quantum_circuit_computation_local(n_qubit=10, n_quanv=3)

=> Number of gates: 65
==================== Gate 0 ====================
====> Dense gate | Pos: 9
State vector after Dense gate:
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
==================== Gate 1 ====================
====> Dense gate | Pos: 9
State vector after Dense gate:
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
==================== Gate 2 ====================
====> Dense gate | Pos: 8
State vector after Dense gate:
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
==================== Gate 3 ====================
====> Dense gate | Pos: 8
State vector after Dense gate:
Complex number: 0j | Hex form: 0000000000000000
Complex number: 0j | Hex form: 0000000000000000
==================== Gate 4 ====================
====> Dense gate | Pos: 7
State vector after Dense gate:
Complex number: (-0.17305647211162345-0.05208923871224047j) | H

In [2]:
quantum_circuit_idx_range = np.arange(1, 20, 1)
qubit_num_idx_range = np.arange(3, 18, 1)

utils.create_folder('./result')

for qubit_num_idx in qubit_num_idx_range:
    saved_folder = './result/quantum_circuit_data_' + str(qubit_num_idx) + '_qubits/'
    utils.create_folder(saved_folder)
    for quantum_circuit_idx in quantum_circuit_idx_range:
        print('===> Processing quantum circuit with ' + str(qubit_num_idx) + ' qubits and ' + str(quantum_circuit_idx) + ' quanvolutional')
        ctx_link = '../hardware/gate_ctx_for_sim/quantum_circuit_data_' + str(qubit_num_idx) + '_qubits/' + 'output_' + str(qubit_num_idx) + '_qubits' + '_quanvolutional_' + str(quantum_circuit_idx) + '.txt'
        state_state = saved_folder + 'output_' + str(qubit_num_idx) + '_qubits' + '_quanvolutional_' + str(quantum_circuit_idx) + '.npy'
        utils.quantum_circuit_res_sim(n_qubit=qubit_num_idx, n_quanv=quantum_circuit_idx, ctx_link=ctx_link, state_link=state_state)


Directory './result' already exists.
Directory './result/quantum_circuit_data_3_qubits/' created successfully.
===> Processing quantum circuit with 3 qubits and 1 quanvolutional
===> Processing quantum circuit with 3 qubits and 2 quanvolutional
===> Processing quantum circuit with 3 qubits and 3 quanvolutional
===> Processing quantum circuit with 3 qubits and 4 quanvolutional
===> Processing quantum circuit with 3 qubits and 5 quanvolutional
===> Processing quantum circuit with 3 qubits and 6 quanvolutional
===> Processing quantum circuit with 3 qubits and 7 quanvolutional
===> Processing quantum circuit with 3 qubits and 8 quanvolutional
===> Processing quantum circuit with 3 qubits and 9 quanvolutional
===> Processing quantum circuit with 3 qubits and 10 quanvolutional
===> Processing quantum circuit with 3 qubits and 11 quanvolutional
===> Processing quantum circuit with 3 qubits and 12 quanvolutional
===> Processing quantum circuit with 3 qubits and 13 quanvolutional
===> Processin

KeyboardInterrupt: 